# Recommender System Evaluations
This lab code is an adaptation of the code below:   
https://colab.research.google.com/github/recohut/notebook/blob/master/_notebooks/2021-07-07-recsys-evaluation-metrics-part-2.ipynb

In [ ]:
import  numpy  as  np
import  pandas  as  pd
import  math

## HR@K

In [ ]:
# For one user:
recommended_list = [156, 1134, 27, 1543, 3345, 143, 32, 533, 11, 43]  #items ids
bought_list = [521, 32, 143, 991]

In [ ]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]
  flags = np.isin(bought_list, recommended_list)
  return (flags.sum() > 0) * 1

In [ ]:
hit_rate_at_k(recommended_list, bought_list, 5)

In [ ]:
hit_rate_at_k(recommended_list, bought_list, 10)

## Precision@K

In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]

  flags = np.isin(bought_list, recommended_list)
  return flags.sum() / len(recommended_list)

In [ ]:
precision_at_k(recommended_list, bought_list, 5)

In [ ]:
precision_at_k(recommended_list, bought_list, 10)

## Recall@K

In [ ]:
def recall_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]

  flags = np.isin(bought_list, recommended_list)
  return flags.sum() / len(bought_list)

In [ ]:
recall_at_k(recommended_list, bought_list, 5)

In [ ]:
recall_at_k(recommended_list, bought_list, 10)

## MAP@K

In [ ]:
# list of 3 users
recommended_list_3_users = [[143,156,1134,991,27,1543,3345,533,11,43],
                            [1134,533,14,4,15,1543,1,99,27,3345],
                            [991,3345,27,533,43,143,1543,156,1134,11]]

bought_list_3_users= [[521,32,143], #user1
                      [143,156,991,43,11], #user2
                      [1,2]] #user3

In [ ]:
def ap_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0

    amount_relevant = len(relevant_indexes)

    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [ ]:
def map_k(recommended_list, bought_list, k=5):

    amount_user = len(bought_list)
    list_ap_k = [ap_k(recommended_list[i], bought_list[i], k) for i in np.arange(amount_user)]

    sum_ap_k = sum(list_ap_k)
    return sum_ap_k/amount_user

In [ ]:
map_k(recommended_list_3_users, bought_list_3_users, 5)

## NDCG@K

In [ ]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    rec = recommended_list
    b = bought_list

    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)

    flags = np.isin(recommended_list, bought_list)
    rank_list = []
    for i in np.arange(len(recommended_list)):
        if i < 2:
            rank_list.append(i+1)
        else:
            rank_list.append(math.log2(i+1))
    if len(recommended_list) == 0:
        return 0
    dcg = sum(np.divide(flags, rank_list)) / len(recommended_list)

    i_dcg = sum(np.divide(1, rank_list)) / len(recommended_list)
    return dcg/i_dcg

In [ ]:
ndcg_at_k(recommended_list, bought_list, 5)

In [ ]:
ndcg_at_k(recommended_list, bought_list, 10)

# Using MS Recommender (msr package)
This lab code is an adaptation of the code below:   
https://github.com/recommenders-team/recommenders/blob/main/examples/03_evaluate/evaluation.ipynb

In [ ]:
# MS Recommender 패키지에서 제공하는 추천시스템 평가함수
from msr.python_evaluation import rmse, mae, map_at_k, ndcg_at_k, precision_at_k, recall_at_k

In [ ]:
COL_USER = "UserId"
COL_ITEM = "MovieId"
COL_RATING = "Rating"
COL_PREDICTION = "Rating"

#### Prepare dummy data

In [ ]:
df_true = pd.DataFrame(
        {
            COL_USER: [1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
            COL_ITEM: [1, 2, 3, 1, 4, 5, 6, 7, 2, 5, 6, 8, 9, 10, 11, 12, 13, 14],
            COL_RATING: [5, 4, 3, 5, 5, 3, 3, 1, 5, 5, 5, 4, 4, 3, 3, 3, 2, 1],
        }
    )
df_pred = pd.DataFrame(
    {
        COL_USER: [1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        COL_ITEM: [3, 10, 12, 10, 3, 5, 11, 13, 4, 10, 7, 13, 1, 3, 5, 2, 11, 14],
        COL_PREDICTION: [14, 13, 12, 14, 13, 12, 11, 10, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5]
    }
)

In [ ]:
df_true

In [ ]:
df_pred

In [ ]:
args = [df_true, df_pred]

#### Rating metrics

In [ ]:
kwargs = dict(col_user=COL_USER, 
              col_item=COL_ITEM, 
              col_rating=COL_RATING, 
              col_prediction=COL_PREDICTION)

eval_rmse = rmse(*args, **kwargs)
eval_mae = mae(*args, **kwargs)

In [ ]:
print(f"RMSE:\t {eval_rmse:f}",
      f"MAE:\t {eval_mae:f}", sep='\n')

#### Ranking metrics

In [ ]:
top_k = 5
kwargs = dict(col_user=COL_USER, 
              col_item=COL_ITEM, 
              col_rating=COL_RATING, 
              col_prediction=COL_PREDICTION, 
              k=top_k)

eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)
eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)

In [ ]:
print(f"Precision@{top_k}:\t {eval_precision:f}",
      f"Recall@{top_k}:\t {eval_recall:f}", 
      f"MAP@{top_k}:\t\t {eval_map:f}",
      f"NDCG@{top_k}:\t {eval_ndcg:f}", sep='\n')

# End